In [1]:
# change current working directory
%cd ..

/Users/alpha/Developer/qtrader


# Week 8

Market Simulation
* **Moments**: evaluation criterion
* **Arbitrage**: evaluation criterion
* **`VAE`**: vanilla variational autoencoder architecure
* **`GAN`**: vanilla generative adversarial network

In [ ]:
import qtrader
# simulation tests
from qtrader.simulation.tests import Arbitrage, Moments

import yaml

import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use('TKAgg')

import warnings
warnings.filterwarnings('ignore')

np.random.seed(0)

In [ ]:
# fetch configuration file
config = yaml.load(open('config/simulation.yaml', 'r'))

## Data Source

In [ ]:
# original returns time-series
try:
    # use cached CSV file
    returns = qtrader.envs.data_loader.Finance.Returns(
        config['tickers'], start_date=config['start_date'], csv=config['csv_file_returns'])
except:
    # download data
    returns = qtrader.envs.data_loader.Finance.Returns(
        config['tickers'], start_date=config['start_date'])
    # cache to CSV file
    returns.to_csv(config['csv_file_returns'])

In [ ]:
# remove infinities and NaN values
returns = qtrader.utils.pandas.clean(returns)

## `AAFT` Simulation

In [ ]:
# generated returns time-series
returns_gen = qtrader.simulation.AAFT(
    returns, random=np.random.normal, random_state=13)

## `VAE` Simulation

In [ ]:
# VAE-friendly data
returns_vae = qtrader.utils.rolling2d(
    returns.values, config['window'])
# get dimensions
num_samples, timesteps, input_dim = returns_vae.shape
# initialize model
vae, encoder, generator = qtrader.contrib.generators.VAE(input_dim,
                                                         timesteps=timesteps,
                                                         batch_size=config['batch_size'],
                                                         intermediate_dim=config['intermediate_dim'],
                                                         latent_dim=config['latent_dim'],
                                                         epsilon_std=1.)

In [ ]:
# fit model
vae.fit(returns_vae, returns_vae, epochs=config['num_epochs'], verbose=0);

In [ ]:
# latent variable
mu = np.zeros(config['latent_dim'])
Sigma = np.diag(np.ones(config['latent_dim']))
Z = np.random.multivariate_normal(mu, Sigma, config['num_samples'])
# generate data
returns_vae_gen = generator.predict(Z, batch_size=config['batch_size'])[:, -1, :]

## Tests

Evaluation metrics for simulated data, comparison basis for generative models.

### Moments



In [ ]:
Moments.run(returns, returns_gen, render=True)

### Statistical Arbitrage



In [ ]:
Arbitrage.run(returns, returns_gen,
              config['tickers'], config['freq'], config['window'], render=True)